## Mark a function

In [1]:
def shared_slot(name):
    def inner(func):
        func.shared_slot = name
        return func
    return inner

## Inspect markings in metaclass

In [2]:
class LangMeta(type):
    def __new__(cls, name, bases, attrs):
        instance = super().__new__(cls, name, bases, attrs)
        print('defining class "{}"'.format(name))
        more_bunch = {}
        for name, val in attrs.items():
            shared_slot = getattr(val, 'shared_slot', None)
            if shared_slot is not None:
                print('found something!')
                more_bunch[shared_slot] = val
        instance.bunch = {**instance.bunch, **more_bunch}
        return instance


class Language(metaclass=LangMeta):
    # current implementation relies on this attribute being at least present
    bunch = {None: lambda: print('dummy')}

l = Language()

l.bunch

defining class Language


## Works with inheritance!

In [20]:
class Clingon(Language):
    
    @shared_slot('foo')
    def funky(self):
        return "hello!"

c = Clingon()

c.bunch

defining class Clingon
found something!


{None: <function __main__.Language.<lambda>()>,
 'foo': <function __main__.Clingon.funky(self)>}

### If we make it a class attribute, all instances will see updates to it!

In [22]:
c2 = Clingon()

c2.bunch['test-me'] = None

c2.bunch

{None: <function __main__.Language.<lambda>()>,
 'foo': <function __main__.Clingon.funky(self)>,
 'test-me': None}

In [23]:
c.bunch

{None: <function __main__.Language.<lambda>()>,
 'foo': <function __main__.Clingon.funky(self)>,
 'test-me': None}

In [30]:
c3 = Clingon()
c3.bunch

{None: <function __main__.Language.<lambda>()>,
 'foo': <function __main__.Clingon.funky(self)>,
 'test-me': None}

At the same time note that `Language.bunch` should not change!

In [24]:
l.bunch

{None: <function __main__.Language.<lambda>()>}

## Deeper Inheritance

In [26]:
class ClingonDialect(Clingon):
    
    @shared_slot('test')
    def jazz(self):
        print('world!')

defining class ClingonDialect
found something!


In [27]:
x = ClingonDialect()

In [28]:
x.bunch

{None: <function __main__.Language.<lambda>()>,
 'foo': <function __main__.Clingon.funky(self)>,
 'test-me': None,
 'test': <function __main__.ClingonDialect.jazz(self)>}

In [29]:
l2 = Language()
l2.bunch

{None: <function __main__.Language.<lambda>()>}